import 程式庫


In [5]:
from database import Database
from get_data import Data
from seperate_company import SeperateCompany
from plot import Plot
from backtest import Backtest
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [6]:
# 建立取得資料物件
data = Data()
# 建立資料分割物件
seperate_company = SeperateCompany()
# 建立畫圖物件
plotter = Plot()

In [7]:
# 測試輸出財報資料    
# factor_dict = data.get('report:EPS,ROE,PB,PS,PE')
factor_dict = data.get('report:EPS,ROE')

In [8]:
# 根據隨入的因子，將公司分成quantile組
# 回測所有quantile
all_result_dict = seperate_company.backtest_all_quantile(factor_dict,quantile=4, start_date="2013-01-01", end_date="2019-01-01",frequency="Q")

In [9]:

# 畫圖
plotter.plot(all_result_dict,index="portfolio_returns",N=0)
# 畫圖
plotter.plot(all_result_dict,index="portfolio_value",N=1)
# 列出DataFrame
plotter.table(all_result_dict,index="portfolio_value")

因子:  EPS


因子:  ROE


因子:  EPS


因子:  ROE


因子:  EPS
              Quantile 1    Quantile 2    Quantile 3    Quantile 4
date                                                              
2013-03-29  9.984989e+05  9.985035e+05  9.985384e+05  9.985525e+05
2013-06-28  1.038748e+06  1.082300e+06  1.158899e+06  9.936953e+05
2013-09-30  1.372056e+06  1.272878e+06  1.168202e+06  1.003491e+06
2013-12-31  1.539616e+06  1.373789e+06  1.153890e+06  9.408378e+05
2014-03-31  1.979516e+06  1.506233e+06  1.332781e+06  1.028573e+06
2014-06-30  2.266858e+06  1.398683e+06  1.244389e+06  1.192340e+06
2014-09-30  2.300934e+06  1.349925e+06  1.158983e+06  1.085534e+06
2014-12-31  2.186097e+06  1.327028e+06  1.152561e+06  1.067617e+06
2015-03-31  2.515262e+06  1.412288e+06  1.210764e+06  1.083620e+06
2015-06-30  2.119954e+06  1.333200e+06  1.158236e+06  1.030926e+06
2015-09-30  2.213468e+06  1.254089e+06  1.015146e+06  9.301557e+05
2015-12-31  2.375373e+06  1.335240e+06  9.482916e+05  9.385907e+05
2016-03-31  2.483398e+06  1.320298e+06  9.434974e+05 

In [1]:
import talib

In [2]:
print(talib.__version__)


0.4.24
